# 載入套件和資料集

In [1]:
import torch
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.svm import SVR
from math import sqrt
from sklearn.tree import DecisionTreeClassifier ,DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import math
from tensorflow.keras import optimizers
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from  sklearn.datasets  import  make_hastie_10_2
from xgboost.sklearn import XGBClassifier
from tqdm import tqdm

import pandas as pd
total_new_student_id = pd.read_csv('C:\\Users\\l8950\\Desktop\\圖書館\\Library_fin.csv',encoding='utf-8',index_col=0)
pd.set_option('display.max_columns',17)
total_new_student_id = total_new_student_id.dropna(how='all')

C:\Users\l8950\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# train_x

In [2]:
#將資料依照學生和學年做排序
total_new_student_id_2 = total_new_student_id.sort_values(by=['學生','學年'])
total_new_student_id_2 = total_new_student_id_2.reset_index(drop=True)
#新增欄位>下學期排名差(因為排序過，所以把排名差的資料全部往上移一個，就是他的下學期排名差)
total_new_student_id_2['下學期排名差'] = total_new_student_id_2["排名差"].shift(-1)
total_new_student_id_2

,學生,學年,年級,學期,概化排名,排名差,系所,必修課程數,選修課程數,學分,借中文書次數,借西方書次數,借醫學書次數,借中文書無對照,借英文書無對照,進出圖書館次數,下學期排名差
0,0003764c18b55548b8e7e0a064fed87fb46fa2a2,100.0,1.0,1.0,5.0,0.0,資訊傳播學系,2.0,2.0,8.0,6.0,0.0,0.0,0.0,0.0,3.0,-1.0
1,0003764c18b55548b8e7e0a064fed87fb46fa2a2,100.0,1.0,2.0,5.0,-1.0,資訊傳播學系,3.0,3.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0,5.0
2,0003764c18b55548b8e7e0a064fed87fb46fa2a2,101.0,2.0,1.0,5.0,5.0,資訊傳播學系,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,42.0,-2.0
3,0003764c18b55548b8e7e0a064fed87fb46fa2a2,101.0,2.0,2.0,5.0,-2.0,資訊傳播學系,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
4,0003764c18b55548b8e7e0a064fed87fb46fa2a2,102.0,3.0,1.0,5.0,2.0,資訊傳播學系,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75774,ffe7fe5d52cd8ca472d8814e189023ba1be3206f,100.0,1.0,2.0,55.0,-43.0,心理學系,7.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0
75775,ffe7fe5d52cd8ca472d8814e189023ba1be3206f,101.0,2.0,1.0,5.0,49.0,心理學系,7.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,15.0,-21.0
75776,ffe7fe5d52cd8ca472d8814e189023ba1be3206f,101.0,2.0,2.0,25.0,-21.0,心理學系,6.0,3.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0
75777,ffe7fe5d52cd8ca472d8814e189023ba1be3206f,102.0,3.0,1.0,5.0,19.0,心理學系,1.0,4.0,17.0,0.0,0.0,0.0,0.0,0.0,10.0,-9.0


In [3]:
#依照學生來刪除重複資料
fliter = total_new_student_id.drop_duplicates(subset='學生')
fliter = fliter.reset_index(drop=True)
fliter = fliter['學生']
print(len(fliter))

12992


In [4]:
#將每個學生的最後一筆資料塞選出來，並刪除
for i in tqdm(range(len(fliter))):
    fliter2 = total_new_student_id_2[total_new_student_id_2['學生']==fliter[i]][-1:].index
    total_new_student_id_2.drop(index=fliter2,inplace=True)
print(len(total_new_student_id_2))

100%|██████████| 12992/12992 [03:26<00:00, 63.02it/s]

62787


In [5]:
#將該資料表的['下學期排名差','學生','系所']刪除，剩下的資料當train_x
total_new_student_id_3 = total_new_student_id_2.drop(['下學期排名差','學生','系所'],axis=1)
total_new_student_id_3 = total_new_student_id_3.reset_index(drop=True)
train_x = total_new_student_id_3
lbaelencoder=LabelEncoder()
train_x

,學年,年級,學期,概化排名,排名差,必修課程數,選修課程數,學分,借中文書次數,借西方書次數,借醫學書次數,借中文書無對照,借英文書無對照,進出圖書館次數
0,100.0,1.0,1.0,5.0,0.0,2.0,2.0,8.0,6.0,0.0,0.0,0.0,0.0,3.0
1,100.0,1.0,2.0,5.0,-1.0,3.0,3.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0
2,101.0,2.0,1.0,5.0,5.0,0.0,2.0,6.0,0.0,0.0,0.0,0.0,0.0,42.0
3,101.0,2.0,2.0,5.0,-2.0,0.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0
4,102.0,3.0,1.0,5.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62782,100.0,1.0,1.0,15.0,0.0,8.0,0.0,20.0,4.0,0.0,0.0,0.0,0.0,7.0
62783,100.0,1.0,2.0,55.0,-43.0,7.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0
62784,101.0,2.0,1.0,5.0,49.0,7.0,0.0,21.0,0.0,0.0,0.0,0.0,0.0,15.0
62785,101.0,2.0,2.0,25.0,-21.0,6.0,3.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0


# train_y

In [6]:
#將資料表的['下學期排名差']當train_y
train_y = total_new_student_id_2['下學期排名差']
train_y = train_y.reset_index(drop=True)
train_y

0        -1.0
1         5.0
2        -2.0
3         2.0
4        -3.0
         ... 
62782   -43.0
62783    49.0
62784   -21.0
62785    19.0
62786    -9.0
Name: 下學期排名差, Length: 62787, dtype: float64

# 模型條參

In [7]:
#把train_x,train_y切割成訓練和驗證
x_train, x_validation, y_train, y_validation = train_test_split(train_x, train_y, test_size=0.2,random_state=42)

# 調整xgboost和RandomForest的參數並查看他們的績效

In [8]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape

XGB = XGBRegressor(n_estimators=100, max_depth=5, min_child_weight=9, gamma=0.01,colsample_bytree=0.8, reg_alpha=0.05, reg_lambda=3, learning_rate=0.02)
XGB.fit(x_train,y_train)
y_pred_XGB = XGB.predict(x_validation)
print("MAE : %f" % mae(y_validation, y_pred_XGB))

MAE : 9.373020


In [9]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
for i in range(1,200,5):
    forest = RandomForestRegressor(random_state=42,max_depth=11,n_estimators=i)
    forest_fit = forest.fit(x_train, y_train)
    y_pred = forest_fit.predict(x_validation)
    print("第"+str(i)+"次")
    print("MAE : %f" % mae(y_validation, y_pred))

第1次
MAE : 10.040738
第6次
MAE : 9.579025
第11次
MAE : 9.524961
第16次
MAE : 9.506309
第21次
MAE : 9.491735
第26次
MAE : 9.474522
第31次
MAE : 9.464869
第36次
MAE : 9.463693
第41次
MAE : 9.461624
第46次
MAE : 9.461156
第51次
MAE : 9.454364
第56次
MAE : 9.455306
第61次
MAE : 9.453818
第66次
MAE : 9.453737
第71次
MAE : 9.451328
第76次
MAE : 9.450909
第81次
MAE : 9.452688
第86次
MAE : 9.452124
第91次
MAE : 9.452298
第96次
MAE : 9.452712
第101次
MAE : 9.452174
第106次
MAE : 9.452680
第111次
MAE : 9.451127
第116次
MAE : 9.450938
第121次
MAE : 9.452249
第126次
MAE : 9.451477
第131次
MAE : 9.451889
第136次
MAE : 9.451973
第141次
MAE : 9.452060
第146次
MAE : 9.451441
第151次
MAE : 9.451858


KeyboardInterrupt: 

In [10]:
# 載入網站提交檔

In [11]:
import pandas as pd
submit_ID = pd.read_csv('C:\\Users\\l8950\\Desktop\\Data_Mining期末\\submit.csv')
print(submit_ID)

                                            學生  排名差
0     003322731d5df9d33d40b43ec5c300f89d956c44  NaN
1     007a91bb90a8bfb3ac4ba999d252a35b824cbb8e  NaN
2     0089869d4d8a20cc6b5ff2627152b9da60343469  NaN
3     00c6deac1fdfde67fb46f73e1fc29ffb48526762  NaN
4     00d718e3d99e68d643d3f76048a567a48249395b  NaN
...                                        ...  ...
2174  ff8a389ebdd2535ed7554a849c2a817ba389be95  NaN
2175  ff9d99962346b12a3649459fdfeaee7e10c25491  NaN
2176  ffa4caec29894fd871e78d68f3cd7f2766c5d6d4  NaN
2177  ffabe81c272b60ea2a34b987f066eb06c111439f  NaN
2178  ffc26298937ceb5121fc50051c83c7aebb0f3588  NaN

[2179 rows x 2 columns]


# test_x

In [12]:
#將Library_fin資料集107學年的資料取出
fliter = (total_new_student_id['學年'] == 107)
test_new_student_id = total_new_student_id[fliter]
test_new_student_id = test_new_student_id.reset_index(drop=True)
#抓出107學年和submit資料集不一樣的學生
test_new_student_id_5 = set(test_new_student_id['學生']).difference(submit_ID['學生'])
test_new_student_id_5 = list(test_new_student_id_5)
test_new_student_id_5 = {'學生':test_new_student_id_5}
print(test_new_student_id_5)
#把不一樣的學生刪除
no_text_x = test_new_student_id[test_new_student_id["學生"].isin(test_new_student_id_5['學生'])]
text_x = test_new_student_id.drop(no_text_x.index)
#並依照加入submit資料集順序加入上學期少的三筆資料，放入text_x
#loss_st_in = [392,490,1439]
loss_st = ['2efbdff86e9162ff381c8b3a6945ec0675bfa8e3','3a17aa229769d2823292bc2a2e970d0d977b078d','a900cfc41530017d24f7d157cad898a4ed94842e']
for i in range(len(loss_st)):
    text_x = text_x.append(total_new_student_id[total_new_student_id['學生']==loss_st[i]][-1:])
text_x = text_x.sort_values(by=['學生'])
text_x = text_x.reset_index(drop=True)
#將text_x資料集中的['學生','系所']刪除，以保持訓練和測試欄位一樣
text_x = text_x.drop(['學生','系所'],axis = 1)
print(text_x)

{'學生': ['e5e2590af1e3dd8fb259632ce993c7facfc92a81', 'd6449e36facf0a1e4d1b9e8ad81dc12cf5dceefe', '53fd3b93845bcbbcbc60e265db92100f1dc3f868', '093b3cacb81b6c1047daf499c04c3b0782695962', '111abb6931fa6d3f91746fed1c6e6a8fbfc7ab97', '126a82f6f992f46edfbefddaa709eab809d0007d', '5b76a0273c317849c12269e21af894d1fe0f62c4', 'f81dc9ebda90940e098f432b9699d275244657fa']}
         學年   年級   學期  概化排名   排名差  必修課程數  選修課程數    學分  借中文書次數  借西方書次數  借醫學書次數  \
0     107.0  3.0  1.0  25.0  -4.0    4.0    7.0  25.0     1.0     0.0     0.0   
1     107.0  3.0  1.0  35.0   6.0    3.0    6.0  21.0     7.0     2.0     0.0   
2     107.0  3.0  1.0  15.0  13.0    2.0    7.0  19.0     0.0     0.0     0.0   
3     107.0  3.0  1.0  15.0   5.0    3.0    4.0  20.0     0.0     0.0     0.0   
4     107.0  3.0  1.0   5.0  13.0    2.0    6.0  18.0     9.0     0.0     0.0   
...     ...  ...  ...   ...   ...    ...    ...   ...     ...     ...     ...   
2174  107.0  3.0  1.0  75.0 -35.0    5.0    0.0  16.0     4.0     0.0   

# 查看是否都一樣

In [13]:
print(train_x.shape)
print(train_y.shape)
print(text_x.shape)

(62787, 14)
(62787,)
(2179, 14)


# 依照上面的模型參數來預測107下學期排名差

In [14]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
#76 5 , min_child_weight=9, gamma=0.01,colsample_bytree=0.8, reg_alpha=0.05, reg_lambda=3, learning_rate=0.02
XGB = XGBRegressor(n_estimators=76, max_depth=5, min_child_weight=9, gamma=0.01,colsample_bytree=0.82, reg_alpha=0.17, reg_lambda=3, learning_rate=0.02,subsample=0.8)
XGB.fit(train_x,train_y)

y_pred_XGB = XGB.predict(text_x)
print(y_pred_XGB.astype(int))
pd.set_option('display.max_columns',10)
pd.set_option('display.max_rows',3000)
pd.set_option('max_colwidth',100)
pd.DataFrame(y_pred_XGB.astype(int),columns =["排名差"])

[ 0 -1 -4 ...  1 -1 -2]


,排名差
0,0
1,-1
2,-4
3,-2
4,-5
5,-3
6,0
7,-1
8,0
9,4


# 將成績存成xgboost_csv檔

In [15]:
print(y_pred_XGB.astype(int))
submit_ID2 = submit_ID.drop(["排名差"],axis=1)
submit_ID2.insert(1,"排名差",y_pred_XGB.astype(int))
print(submit_ID2)
submit_ID2.to_csv("C:\\Users\\l8950\\Desktop\\圖書館\\測試_97.csv",index=False)

[ 0 -1 -4 ...  1 -1 -2]
                                            學生  排名差
0     003322731d5df9d33d40b43ec5c300f89d956c44    0
1     007a91bb90a8bfb3ac4ba999d252a35b824cbb8e   -1
2     0089869d4d8a20cc6b5ff2627152b9da60343469   -4
3     00c6deac1fdfde67fb46f73e1fc29ffb48526762   -2
4     00d718e3d99e68d643d3f76048a567a48249395b   -5
5     00d98d181668cf1e6b19378f9d540a2587ea3726   -3
6     010292328eb39d9d8842cb15c93d6185652460e5    0
7     010338946952be09a66cc3cba1d162724b2e57ee   -1
8     0108689691f37c59a498c52f1f95f59cbcc6e723    0
9     0127cac9bc4da58373fb14fbfdc9e77207c19e0d    4
10    0161594915124f6826a275c06185911a02b2900d   -2
11    017c7d2c2ee9bd47084c12f60bb3824518c6b607    0
12    018db30aed461a2d2e976b9fa8d2ea4c74229547   -4
13    0192ec893f1bbeb923ecbc0a0fceadb0510c9d5f    4
14    01c4583e9a8012f72b152b003d61f808e9585e5e    8
15    021137b7fb37185b65469d8a07e9c549bc735445    0
16    02469978575b1d708a017dd2e1d962bcd52f69f5    7
17    0266a32384036c49072b327d90fe4898d4

# 依照上面的模型參數來預測107下學期排名差

In [16]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape

forest = RandomForestRegressor(random_state=42,max_depth=5,n_estimators=300)
forest_fit = forest.fit(train_x,train_y)
y_pred_RF = forest_fit.predict(text_x)

# 將成績存成RandomForest_csv檔

In [17]:
print(y_pred_RF.astype(int))
submit_ID2 = submit_ID.drop(["排名差"],axis=1)
submit_ID2.insert(1,"排名差",y_pred_RF.astype(int))
print(submit_ID2)
submit_ID2.to_csv("C:\\Users\\l8950\\Desktop\\圖書館\\測試_6_RF.csv",index=False)

[-1 -1 -7 ...  1 -2 -2]
                                            學生  排名差
0     003322731d5df9d33d40b43ec5c300f89d956c44   -1
1     007a91bb90a8bfb3ac4ba999d252a35b824cbb8e   -1
2     0089869d4d8a20cc6b5ff2627152b9da60343469   -7
3     00c6deac1fdfde67fb46f73e1fc29ffb48526762   -3
4     00d718e3d99e68d643d3f76048a567a48249395b   -7
5     00d98d181668cf1e6b19378f9d540a2587ea3726   -6
6     010292328eb39d9d8842cb15c93d6185652460e5    1
7     010338946952be09a66cc3cba1d162724b2e57ee   -2
8     0108689691f37c59a498c52f1f95f59cbcc6e723   -1
9     0127cac9bc4da58373fb14fbfdc9e77207c19e0d    7
10    0161594915124f6826a275c06185911a02b2900d   -2
11    017c7d2c2ee9bd47084c12f60bb3824518c6b607   -1
12    018db30aed461a2d2e976b9fa8d2ea4c74229547   -6
13    0192ec893f1bbeb923ecbc0a0fceadb0510c9d5f    6
14    01c4583e9a8012f72b152b003d61f808e9585e5e   12
15    021137b7fb37185b65469d8a07e9c549bc735445   -1
16    02469978575b1d708a017dd2e1d962bcd52f69f5    9
17    0266a32384036c49072b327d90fe4898d4